In [12]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

from sktime.datasets import load_UCR_UEA_dataset
from tslearn.datasets import UCR_UEA_datasets

#1d interpolation
from scipy.interpolate import interp1d

#tensorboard
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter('runs/ChlorineConcentration')

In [13]:
dataset_name = 'Handwriting'

In [14]:
#load dataset using tslearn
data_loader = UCR_UEA_datasets()
X_train, y_train, X_test, y_test  = data_loader.load_dataset(dataset_name)

In [15]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((150, 152, 3), (150,), (850, 152, 3), (850,))

In [16]:
#encode labels
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

In [17]:
#torch tensor
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

In [21]:
#datasets
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
#dataloaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [22]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

(torch.Size([150, 152, 3]),
 torch.Size([150]),
 torch.Size([850, 152, 3]),
 torch.Size([850]))

In [37]:
#CNN model
class CNN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(CNN, self).__init__()
        self.sequential_1 = nn.Sequential(
            nn.Conv1d(in_channels=input_size, out_channels=16, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            #global average pooling
            nn.AdaptiveAvgPool1d(1)
        )
        self.sequential_2 = nn.Sequential(
            nn.LazyLinear(out_features=128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(in_features=128, out_features=num_classes)
        )
    def forward(self, x):
        #swap axes
        x = x.permute(0, 2, 1)
        x = self.sequential_1(x)
        x = x.reshape(x.shape[0], -1)
        x = self.sequential_2(x)
        return x
#LSM model
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    def forward(self, x):
        #set initial states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        #forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))
        #decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out

In [24]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [38]:
# #cnn model
# input_size = X_train.shape[-1]
# num_classes = len(np.unique(y_train))
# model = CNN(input_size, num_classes).to(device)
# lstm model
input_size = X_train.shape[-1]
hidden_size = 128
num_layers = 2
num_classes = len(np.unique(y_train))
model = LSTM(input_size, hidden_size, num_layers, num_classes).to(device)

In [39]:
#feed a sample through the model
for data, target in train_loader:
    data = data.to(device)
    target = target.to(device)
    output = model(data)
    print(output.shape)
    break

torch.Size([64, 26])


In [40]:
#optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [41]:
#training
n_epochs = 1000
for epoch in range(n_epochs):
    for data, target in train_loader:
        data = data.to(device)
        target = target.to(device)
        #forward
        output = model(data)
        loss = criterion(output, target)
        #backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch: {epoch+1}, Loss: {loss.item():.4f}')

Epoch: 1, Loss: 3.2489
Epoch: 2, Loss: 3.2796
Epoch: 3, Loss: 3.2566
Epoch: 4, Loss: 3.2464
Epoch: 5, Loss: 3.2025
Epoch: 6, Loss: 3.2246
Epoch: 7, Loss: 3.0480
Epoch: 8, Loss: 3.0290
Epoch: 9, Loss: 3.0287
Epoch: 10, Loss: 3.1759
Epoch: 11, Loss: 3.1738
Epoch: 12, Loss: 3.2617
Epoch: 13, Loss: 3.1219
Epoch: 14, Loss: 3.0926
Epoch: 15, Loss: 3.1800
Epoch: 16, Loss: 3.1996
Epoch: 17, Loss: 3.2163
Epoch: 18, Loss: 3.1096
Epoch: 19, Loss: 3.2621
Epoch: 20, Loss: 2.9885
Epoch: 21, Loss: 3.2459
Epoch: 22, Loss: 3.1431
Epoch: 23, Loss: 3.2103
Epoch: 24, Loss: 3.0421
Epoch: 25, Loss: 3.1640
Epoch: 26, Loss: 3.0761
Epoch: 27, Loss: 2.8077
Epoch: 28, Loss: 2.9868
Epoch: 29, Loss: 3.0970
Epoch: 30, Loss: 3.0498
Epoch: 31, Loss: 2.8383
Epoch: 32, Loss: 3.0276
Epoch: 33, Loss: 2.7772
Epoch: 34, Loss: 3.0214
Epoch: 35, Loss: 2.7622
Epoch: 36, Loss: 2.9551
Epoch: 37, Loss: 3.1587
Epoch: 38, Loss: 3.0793
Epoch: 39, Loss: 2.9121
Epoch: 40, Loss: 2.9874
Epoch: 41, Loss: 2.8599
Epoch: 42, Loss: 2.7517
E

KeyboardInterrupt: 

In [42]:
#testing
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for data, target in test_loader:
        data = data.to(device)
        target = target.to(device)
        output = model(data)
        #value, index
        _, predictions = torch.max(output, 1)
        n_samples += target.shape[0]
        n_correct += (predictions == target).sum().item()
    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy: {acc:.2f}')

Accuracy: 9.06
